In [2]:
import sys
sys.executable

'd:\\1.VINA\\ai_project\\milvus_ai_local\\.venv\\Scripts\\python.exe'

In [4]:
from pymilvus import (
    connections, FieldSchema, CollectionSchema,
    DataType, Collection, utility
)
import pandas as pd
from sentence_transformers import SentenceTransformer

In [5]:
connections.connect(
    alias="default",
    host="10.50.171.40",
    port="19530"
)

In [4]:
#check connection milvus
utility.list_collections()

['kb_docs', 'kb_docs_v2_model3']

**Just For Flushing the Dataset**

In [7]:
df = pd.read_excel(r"D:\\1.VINA\\ai_project\\Issue_Recap_2025.xlsx")
df = df.dropna(subset=["Troubleshoot Step"])
df.drop(columns=['Month', 'Created By', 'Quarter'], inplace=True)

df.head()

,Ticket Number,Ticket Summary,Type Ticket,Category,Sub Category,Department,Assign Name,Troubleshoot Step
0,11358012025,Penghapusan data IT Inventory Mutasi barang ja...,SERP,Custom Report SERP,Custom Existing Report SERP,CUSTOM,Mumu Mukhlas,1. Checking Custom Report 112 and 111 then com...
1,11394012025,Report 112 minus sedangkan system sudah close ...,JIT,Troubleshooting JIT,OTHER JIT,SCM,Iman Suryadi,1. Check daily transaction base on finding min...
2,11408012025,Pada module production report > prod monthly r...,JIT,Custom Report JIT,Custom New Report JIT,SCM,Iman Suryadi,1. Raise up to XC for add technology/dept show...
4,11470012025,tidak bisa save LPB material IA,SERP,Troubleshooting SERP,Raw Materials Warehouse,PPIC,Mumu Mukhlas,"1. Checking Custom Report 108, 110 and 112 the..."
5,11549012025,Assign Acc Accrued New Transaction on Dec (Bal...,Yonsuite,Custom Report Yonsuite,Custom Existing Report Yonsuite,FA,Dea Aldiani,1. Add new COAs in report item \n2. Recalculat...


In [8]:
def build_embedding_text(row):
    return f"""
Issue:
{row['Ticket Summary']}

Troubleshooting Steps:
{row['Troubleshoot Step']}
""".strip()

texts = df.apply(build_embedding_text, axis=1).tolist()
ids = df.index.astype(int).tolist()

In [9]:
#Checking the text concat result
texts[0]

'Issue:\nPenghapusan data IT Inventory Mutasi barang jadi dan Mutasi Scrap\n\nTroubleshooting Steps:\n1. Checking Custom Report 112 and 111 then compare with data on IT Inventory,\n2. Info to DOE Team via mail for Remove the datat ransaction on IT Inventory'

In [13]:
#Embedding function using ollama

import requests

OLLAMA_URL = "http://localhost:11434"
EMBED_MODEL = "nomic-embed-text"

def embed_with_ollama(texts):
    embeddings = []

    for text in texts:
        resp = requests.post(
            f"{OLLAMA_URL}/api/embeddings",
            json={
                "model": EMBED_MODEL,
                "prompt": text
            },
            timeout=60
        )
        resp.raise_for_status()
        embeddings.append(resp.json()["embedding"])

    return embeddings

In [11]:
#Testing Embedding Function
test_vec = embed_with_ollama([texts[0]])

len(test_vec[0])

768

In [12]:
#Embed all KB records
embeddings = embed_with_ollama(texts)

len(embeddings), len(embeddings[0])

(325, 768)

In [15]:
#Create New Collection

COLLECTION_NAME = "helpdesk_kb"

if not utility.has_collection(COLLECTION_NAME):
    fields = [
    FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=True
    ),
    FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=768
    ),
    FieldSchema(
        name="ticket_number",
        dtype=DataType.VARCHAR,
        max_length=12
    ),
    FieldSchema(
        name="ticket_summary",
        dtype=DataType.VARCHAR,
        max_length=1024
    ),
    FieldSchema(
        name="type_ticket",
        dtype=DataType.VARCHAR,
        max_length=64
    ),
    FieldSchema(
        name="category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="sub_category",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="department",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="assign_name",
        dtype=DataType.VARCHAR,
        max_length=128
    ),
    FieldSchema(
        name="troubleshoot_step",
        dtype=DataType.VARCHAR,
        max_length=4096
    ),
    ]

    schema = CollectionSchema(
        fields=fields,
        description="Helpdesk KB (Ollama embeddings)"
    )

    collection = Collection(
            name=COLLECTION_NAME,
            schema=schema
    )

    print("Collection created:", COLLECTION_NAME)
else:
    collection = Collection(COLLECTION_NAME)
    print("Collection already exists:", COLLECTION_NAME)

Collection created: helpdesk_kb


In [16]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "IP",   # inner product (best for Ollama)
    "params": {"nlist": 128}
}

collection.create_index(
    field_name="embedding",
    index_params=index_params
)

Status(code=0, message=)

In [8]:
collection.load()

In [20]:
data_to_insert = [
    embeddings,
    df["Ticket Number"].astype(str).tolist(),
    df["Ticket Summary"].tolist(),
    df["Type Ticket"].tolist(),
    df["Category"].tolist(),
    df["Sub Category"].astype(str).tolist(),
    df["Department"].tolist(),
    df["Assign Name"].tolist(),
    df["Troubleshoot Step"].tolist()
]

collection.insert(data_to_insert)
collection.flush()

In [15]:
#checking the flush
COLLECTION_NAME = 'helpdesk_kb'
collection = Collection(COLLECTION_NAME)
collection.num_entities

325

In [16]:
def score_to_percent(score, min_score=0.3, max_score=0.95):
    """
    Convert Milvus IP score to confidence percentage (display only)
    """
    score = max(min(score, max_score), min_score)
    return round((score - min_score) / (max_score - min_score) * 100, 2)

In [17]:
#Testing the environment

query = "Perubahan reason"

query_vector = embed_with_ollama([query])

results = collection.search(
    data=query_vector,
    anns_field="embedding",
    param={
        "metric_type": "IP",
        "params": {"nprobe": 10}
    },
    limit=3,
    output_fields=["troubleshoot_step"]
)

for hit in results[0]:
    percent = score_to_percent(hit.score)
    print(f"Confidence: {percent}%")
    print("Raw score:", round(hit.score, 4))
    print(hit.entity.get("troubleshoot_step"))
    print("-" * 40)

Confidence: 100.0%
Raw score: 296.1676
follow up to GOCT SDI (Eric)
----------------------------------------
Confidence: 100.0%
Raw score: 294.3394
follow up to GOCT SDI (Eric)
----------------------------------------
Confidence: 100.0%
Raw score: 292.1916
1. Try to Input Request
2. System Issue because the pop up message appear, need Rise Up to SDI XC Team
----------------------------------------
